In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import requests
import os
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from pathlib import Path
from pandas.errors import ParserError

def _download_worker(args):
    """
    Downloads a single image, handles errors, and skips existing files.
    This is the helper function for our main downloader.
    """
    image_url, save_path = args

    # Skip if we already have this image
    if os.path.exists(save_path):
        return 'skipped'

    try:
        # Use a timeout to avoid getting stuck on a bad link
        response = requests.get(image_url, stream=True, timeout=20)
        # Raise an error if the link is broken (like a 404 Not Found)
        response.raise_for_status()

        # Save the image to the specified path
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return 'success'
    except requests.exceptions.RequestException:
        # If anything goes wrong, just mark it as failed and move on
        return 'failed'

def download_images_from_csv(csv_path: str, image_link_column: str, filename_column: str, save_folder: str, chunksize: int = 10000):
    """
    Reads a CSV file in chunks, handling ParserError, and downloads all images.

    Args:
        csv_path (str): The path to your CSV file in Colab.
        image_link_column (str): The name of the column containing the image URLs.
        filename_column (str): The column to use for naming the saved files (e.g., 'sample_id').
        save_folder (str): The name of the folder where images will be saved.
        chunksize (int): The number of rows to read at a time from the CSV.
    """
    print(f"Attempting to read image links from '{csv_path}' in chunks...")

    df_list = []
    try:
        # Wrap the iterator creation in try-except
        csv_iterator = pd.read_csv(csv_path, chunksize=chunksize, low_memory=False)
        for i, chunk in enumerate(csv_iterator):
            df_list.append(chunk)
    except FileNotFoundError:
        print(f"--- ERROR: The file '{csv_path}' was not found. Please upload it first. ---")
        return
    except ParserError as e:
        print(f"--- WARNING: ParserError encountered while setting up or reading a chunk. Processing may be incomplete. Error: {e} ---")
        # If the iterator creation or first chunk read fails, df_list might be empty or incomplete
        # We will proceed with whatever was successfully read, if anything.
        pass # Allow processing of successfully read chunks if any

    if not df_list:
        print("--- No valid data chunks were read from the CSV. ---")
        return

    df = pd.concat(df_list, ignore_index=True)
    print(f"Successfully read {len(df)} rows from the CSV (potentially with skipped chunks).")

    # Create the folder to save images in, if it doesn't exist
    os.makedirs(save_folder, exist_ok=True)
    print(f"Images will be saved in the '{save_folder}' directory.")

    # Create a list of all download jobs
    tasks = []
    for _, row in df.iterrows():
        image_url = row[image_link_column]
        # Use the filename_column to create a unique filename, adding .jpg
        filename = f"{row[filename_column]}.jpg"
        full_save_path = os.path.join(save_folder, filename)
        tasks.append((image_url, full_save_path))

    # Use ThreadPoolExecutor to download images in parallel (much faster!)
    print(f"Starting download of {len(tasks)} images...")
    with ThreadPoolExecutor(max_workers=32) as executor:
        results = list(tqdm(executor.map(_download_worker, tasks), total=len(tasks)))

    # Print a final summary
    success_count = results.count('success')
    skipped_count = results.count('skipped')
    failed_count = results.count('failed')
    print("\n--- Download Complete! ---")
    print(f"  Successfully downloaded: {success_count}")
    print(f"  Already existed (skipped): {skipped_count}")
    print(f"  Failed to download:      {failed_count}")
    print("--------------------------")

In [ ]:
# Cloud Storage
from google.cloud import storage
storage_client = storage.Client(project='notebooked61a9156c')


In [ ]:
# 1. Define your file and column names
my_csv_file = '/kaggle/input/challenge/test.csv'
url_column_name = 'image_link'
id_column_name = 'sample_id'
output_folder_name = 'test_images'

# 2. Run the updated function
download_images_from_csv(
    csv_path=my_csv_file,
    image_link_column=url_column_name,
    filename_column=id_column_name,
    save_folder=output_folder_name
)

In [ ]:
# 1. Define your file and column names
my_csv_file = '/kaggle/input/challenge/train.csv'
url_column_name = 'image_link'
id_column_name = 'sample_id'
output_folder_name = 'train_images'

# 2. Run the updated function
download_images_from_csv(
    csv_path=my_csv_file,
    image_link_column=url_column_name,
    filename_column=id_column_name,
    save_folder=output_folder_name
)

In [ ]:
import numpy as np
import pandas as pd
import os

# --- Make sure your DataFrames are loaded ---
train_df = pd.read_csv('/kaggle/input/challenge/train.csv')
test_df = pd.read_csv('/kaggle/input/challenge/test.csv')
# --- 1. IMPORTANT: EDIT THIS LINE ---
# Replace 'your-dataset-name-here' with the actual name of your dataset folder.
# You can find this in the "Data" panel on the right side of your notebook.
input_path = '/kaggle/input/challenge'

# --- 2. Verify that the path is correct ---
print(f"Looking for files in: {input_path}")
print("Files found:", os.listdir(input_path))


# --- 3. Build the full paths to your embedding files ---
path_train_text = os.path.join(input_path, 'train_text_embeddings.npy')
path_test_text = os.path.join(input_path, 'test_text_embeddings.npy')
path_train_image = os.path.join(input_path, 'train_image_embeddings.npy')
path_test_image = os.path.join(input_path, 'test_image_embeddings.npy')

# --- 4. Load all embeddings using the correct full paths ---
try:
    print("\nLoading all saved embeddings...")
    train_text_embeddings = np.load(path_train_text)
    test_text_embeddings = np.load(path_test_text)
    train_image_embeddings = np.load(path_train_image)
    test_image_embeddings = np.load(path_test_image)

    print("\n✅ All embeddings loaded successfully!")
    print(f"Shape of train_text_embeddings: {train_text_embeddings.shape}")
    print(f"Shape of train_image_embeddings: {train_image_embeddings.shape}")

except FileNotFoundError as e:
    print(f"\n--- ERROR ---")
    print(f"A file was not found. Please double-check your input_path.")
    print(f"Original error: {e}")



# --- Re-create the item_quantity feature (if needed) ---
import re
def extract_ipq(text):
    match = re.search(r'(?:IPQ|Pack of|Count|pack-of|pack|Pack|count)[\s:]*(\d+)', str(text), re.IGNORECASE)
    return int(match.group(1)) if match else 1

train_df['item_quantity'] = train_df['catalog_content'].apply(extract_ipq)
test_df['item_quantity'] = test_df['catalog_content'].apply(extract_ipq)

# --- Horizontally stack (combine) all features into one matrix ---
print("Combining all features into final training and test sets...")
X_train = np.hstack([
    train_df[['item_quantity']].values,
    train_text_embeddings,
    train_image_embeddings
])

X_test = np.hstack([
    test_df[['item_quantity']].values,
    test_text_embeddings,
    test_image_embeddings
])

# --- Define the target variable with log transformation ---
# This helps the model optimize for the SMAPE metric.
y_train = np.log1p(train_df['price'])

print(f"\n✅ Final training data shape: {X_train.shape}")
print(f"✅ Final test data shape: {X_test.shape}")

In [ ]:
import numpy as np
import os

# --- 1. IMPORTANT: EDIT THIS LINE ---
# Replace 'your-dataset-name-here' with the actual name of your dataset folder.
# You can find this in the "Data" panel on the right side of your notebook.
input_path = '/kaggle/input/challenge'

# --- 2. Verify that the path is correct ---
print(f"Looking for files in: {input_path}")
print("Files found:", os.listdir(input_path))


# --- 3. Build the full paths to your embedding files ---
path_train_text = os.path.join(input_path, 'train_text_embeddings.npy')
path_test_text = os.path.join(input_path, 'test_text_embeddings.npy')
path_train_image = os.path.join(input_path, 'train_image_embeddings.npy')
path_test_image = os.path.join(input_path, 'test_image_embeddings.npy')


# --- 4. Load all embeddings using the correct full paths ---
try:
    print("\nLoading all saved embeddings...")
    train_text_embeddings = np.load(path_train_text)
    test_text_embeddings = np.load(path_test_text)
    train_image_embeddings = np.load(path_train_image)
    test_image_embeddings = np.load(path_test_image)

    print("\n✅ All embeddings loaded successfully!")
    print(f"Shape of train_text_embeddings: {train_text_embeddings.shape}")
    print(f"Shape of train_image_embeddings: {train_image_embeddings.shape}")

except FileNotFoundError as e:
    print(f"\n--- ERROR ---")
    print(f"A file was not found. Please double-check your input_path.")
    print(f"Original error: {e}")

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold

# --- K-Fold Cross-Validation Setup ---
NFOLDS = 5
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=42)

# Arrays to store predictions
oof_predictions = np.zeros(X_train.shape[0]) # Predictions on our own training data
test_predictions = np.zeros(X_test.shape[0]) # Predictions for the submission file

# --- Training Loop ---
for fold, (train_index, val_index) in enumerate(kf.split(X_train, y_train)):
    print(f"========== FOLD {fold+1} ==========")
    x_train_fold, x_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    model = lgb.LGBMRegressor(
        objective='regression_l1',  # MAE is a good objective for SMAPE
        n_estimators=2000,
        learning_rate=0.05,
        num_leaves=32,
        n_jobs=-1,
        seed=42
    )

    model.fit(x_train_fold, y_train_fold,
              eval_set=[(x_val_fold, y_val_fold)],
              eval_metric='rmse',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    # Store predictions
    oof_predictions[val_index] = model.predict(x_val_fold)
    test_predictions += model.predict(X_test) / NFOLDS # Average test predictions over the 5 folds

print("\n✅ Model training complete!")

In [ ]:
# --- Inverse transform the log predictions to get actual prices ---
final_predictions = np.expm1(test_predictions)

# --- Create the submission DataFrame ---
submission_df = pd.DataFrame({
    'sample_id': test_df['sample_id'],
    'price': final_predictions
})

# Ensure all prices are positive, as required
submission_df['price'] = submission_df['price'].clip(lower=0.01)

# --- Save the final file ---
submission_df.to_csv('test_out.csv', index=False)

print("🎉 Submission file 'test_out.csv' created successfully!")
print("Here's a preview of your submission file:")
print(submission_df.head())

In [ ]:
# --- 1. Define the SMAPE function ---
def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    # Handle the edge case where both true and pred are zero
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

# --- 2. Get the true prices from the original dataframe ---
actual_prices = train_df['price'].values

# --- 3. IMPORTANT: Convert OOF predictions from log scale back to normal prices ---
# This is a critical step! The model predicted log(price+1).
oof_prices = np.expm1(oof_predictions)

# --- 4. Calculate and print your estimated score ---
local_smape_score = smape(actual_prices, oof_prices)

print(f"Your estimated Out-of-Fold SMAPE score is: {local_smape_score:.4f}%")

In [ ]:
# List all files in your "desk" (the output directory)
!ls /kaggle/working/

In [ ]:
import numpy as np
import pandas as pd

# --- 1. Define the SMAPE function ---
def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    # Handle the edge case where both true and pred are zero to avoid division by zero
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

# --- 2. Load the training data to get the true prices ---
# Make sure the path to your train.csv is correct
# For Kaggle, it would be something like '/kaggle/input/your-dataset-name/train.csv'
train_df = pd.read_csv('/kaggle/working/test_out.csv')

actual_prices = train_df['price'].values

# --- 3. IMPORTANT: Convert OOF predictions back to the price scale ---
# The model predicted log(price+1), so we need to reverse that.
oof_prices = np.expm1(oof_predictions)

# --- 4. Calculate and print your estimated score ---
local_smape_score = smape(actual_prices, oof_prices)

print("="*50)
print(f"Your estimated Out-of-Fold SMAPE score is: {local_smape_score:.4f}%")
print("="*50)

In [ ]:
import pandas as pd
import numpy as np
import re

# --- Load your original CSV files ---
# Make sure the path is correct for your environment (e.g., Kaggle, Colab)
try:
    train_df = pd.read_csv('/kaggle/input/challenge/train.csv')
    test_df = pd.read_csv('/kaggle/input/challenge/test.csv')
    print("✅ CSV files loaded successfully.")
except FileNotFoundError:
    print("--- ERROR: train.csv or test.csv not found. Please check the path. ---")
    # You might need to add the full path, e.g., '/kaggle/input/your-dataset/train.csv'

# --- Feature 1: Extract Brand Name (High Impact) ---
def find_brand(text):
    text = str(text).lower()
    # Add more brands as you discover them in your data
    brands = ['apple', 'samsung', 'sony', 'nike', 'dell', 'hp', 'lego', 'adidas']
    for brand in brands:
        if brand in text:
            return brand
    return 'unknown'

print("Extracting brand names...")
train_df['brand'] = train_df['catalog_content'].apply(find_brand)
test_df['brand'] = test_df['catalog_content'].apply(find_brand)

# --- Feature 2: Extract Numerical Specs ---
print("Extracting numerical specifications...")
# Extracts numbers followed by 'GB' (for memory/storage)
train_df['spec_gb'] = train_df['catalog_content'].str.extract(r'(\d+)\s*GB', flags=re.IGNORECASE).astype(float).fillna(0)
test_df['spec_gb'] = test_df['catalog_content'].str.extract(r'(\d+)\s*GB', flags=re.IGNORECASE).astype(float).fillna(0)

# Extracts numbers followed by 'ml' (for liquids)
train_df['spec_ml'] = train_df['catalog_content'].str.extract(r'(\d+)\s*ml', flags=re.IGNORECASE).astype(float).fillna(0)
test_df['spec_ml'] = test_df['catalog_content'].str.extract(r'(\d+)\s*ml', flags=re.IGNORECASE).astype(float).fillna(0)

# --- Convert the 'brand' column from text to numbers ---
print("Encoding categorical 'brand' feature...")
train_brands_encoded = pd.get_dummies(train_df['brand'], prefix='brand')
test_brands_encoded = pd.get_dummies(test_df['brand'], prefix='brand')

# Align columns to ensure both have the same brand features
train_brands_encoded, test_brands_encoded = train_brands_encoded.align(test_brands_encoded, join='left', axis=1, fill_value=0)

print("\n✅ New features created and encoded successfully!")
print("Here's a sample of the new features:")
print(train_df[['brand', 'spec_gb', 'spec_ml']].head())

In [ ]:
# This assumes your original X_train and X_test matrices are already in memory.
# If not, you'll need to load them from your .npy files first.

print("Combining original features with new features...")
X_train_new = np.hstack([
    X_train,  # Your original 897 features
    train_brands_encoded.values,
    train_df[['spec_gb', 'spec_ml']].values
])

X_test_new = np.hstack([
    X_test,   # Your original 897 features
    test_brands_encoded.values,
    test_df[['spec_gb', 'spec_ml']].values
])

print(f"\nOld number of features: {X_train.shape[1]}")
print(f"New number of features: {X_train_new.shape[1]}")

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

# --- K-Fold Cross-Validation Setup ---
NFOLDS = 5
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=42)

# --- Use NEW variable names to store predictions from our new model ---
oof_predictions_new = np.zeros(X_train_new.shape[0])
test_predictions_new = np.zeros(X_test_new.shape[0])

# --- Training Loop ---
# This will train 5 separate models on your new, feature-rich dataset
for fold, (train_index, val_index) in enumerate(kf.split(X_train_new, y_train)):
    print(f"========== FOLD {fold+1} ==========")
    # Use the NEW data matrices for training and validation
    x_train_fold, x_val_fold = X_train_new[train_index], X_train_new[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the LightGBM model (same settings as before for a fair comparison)
    model = lgb.LGBMRegressor(
        objective='regression_l1',
        n_estimators=2000,
        learning_rate=0.05,
        num_leaves=32,
        n_jobs=-1,
        seed=42,
        boosting_type='gbdt',
    )

    # Train the model
    model.fit(x_train_fold, y_train_fold,
              eval_set=[(x_val_fold, y_val_fold)],
              eval_metric='rmse',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    # Store the new predictions
    oof_predictions_new[val_index] = model.predict(x_val_fold)
    test_predictions_new += model.predict(X_test_new) / NFOLDS

print("\n✅ New model training complete!")

In [ ]:
# --- Define the SMAPE function ---
def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

# --- Get the true prices ---
actual_prices = train_df['price'].values

# --- Convert NEW OOF predictions back to the price scale ---
oof_prices_new = np.expm1(oof_predictions_new)

# --- Calculate and print your new estimated score ---
local_smape_score_new = smape(actual_prices, oof_prices_new)

print("\n" + "="*50)
print(f"Your NEW estimated SMAPE score is: {local_smape_score_new:.4f}%")
print("="*50)

In [ ]:
# This uses the new OOF predictions you just generated
error_df = pd.DataFrame({
    'actual_price': np.expm1(y_train),
    'predicted_price': np.expm1(oof_predictions_new),
    'catalog_content': train_df['catalog_content']
})
error_df['error'] = np.abs(error_df['actual_price'] - error_df['predicted_price'])

# Find your top 10 worst predictions
worst_errors = error_df.sort_values(by='error', ascending=False)

print("Top 10 Worst Predictions:")
print(worst_errors.head(10))

In [ ]:
import pandas as pd
import numpy as np
import re

# This assumes train_df and test_df are already loaded with your previous features

print("Creating new advanced features...")

# --- Feature 1: A flag for bulk keywords ---
bulk_keywords = ['kit', 'pallet', 'case', 'bucket', 'pack', 'bulk', 'servings', 'supply', 'bottles']
def check_for_bulk(text):
    text = str(text).lower()
    for keyword in bulk_keywords:
        if keyword in text:
            return 1
    return 0

train_df['is_bulk'] = train_df['catalog_content'].apply(check_for_bulk)
test_df['is_bulk'] = test_df['catalog_content'].apply(check_for_bulk)

# --- Feature 2: A better way to get quantity (servings) ---
def get_servings(text):
    text = str(text).lower()
    match = re.search(r'(\d+)\s*servings', text)
    if match:
        return int(match.group(1))
    return 0

train_df['total_servings'] = train_df['catalog_content'].apply(get_servings)
test_df['total_servings'] = test_df['catalog_content'].apply(get_servings)

# --- Feature 3: Standardized Weight ---
def get_weight_in_grams(text):
    text = str(text).lower()
    # Pounds to grams (1 lb = 453.592g)
    match_lb = re.search(r'(\d+\.?\d*)\s*(lb|pound)', text)
    if match_lb:
        return float(match_lb.group(1)) * 453.592
    
    # Ounces to grams (1 oz = 28.35g)
    match_oz = re.search(r'(\d+\.?\d*)\s*oz', text)
    if match_oz:
        return float(match_oz.group(1)) * 28.35
    
    return 0

train_df['weight_grams'] = train_df['catalog_content'].apply(get_weight_in_grams)
test_df['weight_grams'] = test_df['catalog_content'].apply(get_weight_in_grams)

print("✅ New forensic features created!")

In [ ]:
# This assumes X_train_new and X_test_new from your 56.1 model are in memory
new_forensic_features_train = train_df[['is_bulk', 'total_servings', 'weight_grams']].values
new_forensic_features_test = test_df[['is_bulk', 'total_servings', 'weight_grams']].values

# Combine everything
X_train_final = np.hstack([X_train_new, new_forensic_features_train])
X_test_final = np.hstack([X_test_new, new_forensic_features_test])

print(f"Final number of features: {X_train_final.shape[1]}")

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

# --- K-Fold Cross-Validation Setup ---
NFOLDS = 5
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=42)

# --- Use FINAL variable names to store the last set of predictions ---
oof_predictions_final = np.zeros(X_train_final.shape[0])
test_predictions_final = np.zeros(X_test_final.shape[0])

# --- Training Loop on the Final Dataset ---
for fold, (train_index, val_index) in enumerate(kf.split(X_train_final, y_train)):
    print(f"========== FOLD {fold+1} ==========")
    # Use the FINAL data matrices for training
    x_train_fold, x_val_fold = X_train_final[train_index], X_train_final[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    model = lgb.LGBMRegressor(
        objective='regression_l1',
        n_estimators=2000,
        learning_rate=0.05,
        num_leaves=32,
        n_jobs=-1,
        seed=42,
        boosting_type='gbdt',
    )

    model.fit(x_train_fold, y_train_fold,
              eval_set=[(x_val_fold, y_val_fold)],
              eval_metric='rmse',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    # Store the final predictions
    oof_predictions_final[val_index] = model.predict(x_val_fold)
    test_predictions_final += model.predict(X_test_final) / NFOLDS

print("\n✅ Final model training complete!")

In [ ]:
# --- Define the SMAPE function ---
def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

# --- Get the true prices ---
actual_prices = train_df['price'].values

# --- Convert FINAL OOF predictions back to the price scale ---
oof_prices_final = np.expm1(oof_predictions_final)

# --- Calculate and print your new estimated score ---
local_smape_score_final = smape(actual_prices, oof_prices_final)

print("\n" + "="*50)
print(f"Your FINAL estimated SMAPE score is: {local_smape_score_final:.4f}%")
print("="*50)

In [ ]:
pip install xgboost

In [ ]:
model = xgb.XGBRegressor(
    # ... other settings ...
    tree_method='hist',      # <-- Change this to the standard 'hist'
    device='cuda',           # <-- Add this line to explicitly use the GPU
    # ... other settings ...
)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

model = xgb.XGBRegressor(
    # ... other settings ...
    tree_method='hist',      # <-- Change this to the standard 'hist'
    device='cuda',           # <-- Add this line to explicitly use the GPU
    # ... other settings ...
)

# --- K-Fold Cross-Validation Setup ---
NFOLDS = 5
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=42)

# --- Use new variable names for the XGBoost predictions ---
oof_predictions_xgb = np.zeros(X_train_final.shape[0])
test_predictions_xgb = np.zeros(X_test_final.shape[0])

# --- Training Loop for XGBoost ---
for fold, (train_index, val_index) in enumerate(kf.split(X_train_final, y_train)):
    print(f"========== FOLD {fold+1} ==========")
    x_train_fold, x_val_fold = X_train_final[train_index], X_train_final[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the XGBoost model
    model = xgb.XGBRegressor(
        objective='reg:squarederror', # Standard objective for regression
        n_estimators=2000,
        learning_rate=0.05,
        tree_method='hist',      # <-- CHANGE THIS
        device='cuda',           # <-- ADD THIS LINE
  # VERY IMPORTANT: This tells XGBoost to use the GPU
        early_stopping_rounds=100, # Equivalent to LightGBM's early stopping
        n_jobs=-1,
        seed=42
    )

    # Train the model
    model.fit(x_train_fold, y_train_fold,
              eval_set=[(x_val_fold, y_val_fold)],
              verbose=False) # Hides the training progress for each iteration

    # Store the XGBoost predictions
    oof_predictions_xgb[val_index] = model.predict(x_val_fold)
    test_predictions_xgb += model.predict(X_test_final) / NFOLDS

print("\n✅ XGBoost model training complete!")

In [ ]:
import numpy as np
import pandas as pd

# --- Step 1: Define the SMAPE function ---
def smape(y_true, y_pred):
    """Calculates the Symmetric Mean Absolute Percentage Error (SMAPE)."""
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

# --- Step 2: Load the training data to get the true prices ---
# Make sure the path to your train.csv is correct for your environment.
try:
    train_df = pd.read_csv('/kaggle/input/challenge/train.csv')
    actual_prices = train_df['price'].values
    print("✅ Training data loaded successfully.")
except FileNotFoundError:
    print("--- ERROR: 'train.csv' not found. Please check the file path. ---")
    assert False, "File not found. Correct the path."

# ===================================================================
# IMPORTANT: This assumes 'oof_predictions_final' (from LightGBM) and 
# 'oof_predictions_xgb' (from XGBoost) exist in your notebook's memory.
# ===================================================================

# --- Step 3: Recalculate the LightGBM score to define the variable ---
print("Calculating score for LightGBM model...")
oof_prices_final = np.expm1(oof_predictions_final) # <-- From your LGBM model
local_smape_score_final = smape(actual_prices, oof_prices_final)

# --- Step 4: Calculate the XGBoost score ---
print("Calculating score for XGBoost model...")
oof_prices_xgb = np.expm1(oof_predictions_xgb) # <-- From your XGBoost model
local_smape_score_xgb = smape(actual_prices, oof_prices_xgb)

# --- Step 5: Print the final comparison ---
print("\n" + "="*50)
print(f"Your LightGBM (Final) Score was: {local_smape_score_final:.4f}%")
print(f"Your NEW XGBoost Score is:        {local_smape_score_xgb:.4f}%")
print("="*50)

In [ ]:
# --- Blend the Out-of-Fold (OOF) predictions to check the ensemble score ---
# This assumes 'oof_predictions_final' (from LGBM) and 'oof_predictions_xgb' are in memory.
oof_predictions_ensemble = (oof_predictions_final * 0.5) + (oof_predictions_xgb * 0.5)

# Convert the blended log predictions back to the price scale
oof_prices_ensemble = np.expm1(oof_predictions_ensemble)

# Calculate the final score for the ensemble
local_smape_score_ensemble = smape(actual_prices, oof_prices_ensemble)

print("\n" + "="*50)
print(f"Your LightGBM Score:       {local_smape_score_final:.4f}%")
print(f"Your XGBoost Score:        {local_smape_score_xgb:.4f}%")
print(f"Your FINAL ENSEMBLE Score is: {local_smape_score_ensemble:.4f}%")
print("="*50)

In [ ]:
# --- Blend the final test predictions for the submission file ---
# This assumes 'test_predictions_final' (from LGBM) and 'test_predictions_xgb' are in memory.
test_predictions_ensemble = (test_predictions_final * 0.5) + (test_predictions_xgb * 0.5)

# Inverse transform the blended test predictions
final_predictions_ensemble = np.expm1(test_predictions_ensemble)

# Create the new submission DataFrame
submission_df_ensemble = pd.DataFrame({
    'sample_id': test_df['sample_id'],
    'price': final_predictions_ensemble
})

# Ensure all prices are positive
submission_df_ensemble['price'] = submission_df_ensemble['price'].clip(lower=0.01)

# Save the final file
submission_df_ensemble.to_csv('test_out_ensemble.csv', index=False)

print("\n🎉 Final ensemble submission file 'test_out_ensemble.csv' created successfully!")
print("Here's a preview:")
print(submission_df_ensemble.head())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# We will look for the 2000 most important single words, word pairs, and triplets
tfidf = TfidfVectorizer(
    ngram_range=(1, 3), # (single words, pairs, triplets)
    max_features=2000,
    stop_words='english'
)

print("Fitting TF-IDF on training data...")
X_train_tfidf = tfidf.fit_transform(train_df['catalog_content'].fillna(''))

print("Transforming test data with TF-IDF...")
X_test_tfidf = tfidf.transform(test_df['catalog_content'].fillna(''))

# --- Rebuild your FINAL dataset, now with TF-IDF features ---
# Convert the sparse TF-IDF matrix to a dense array for hstack
X_train_final_v2 = np.hstack([X_train_final, X_train_tfidf.toarray()])
X_test_final_v2 = np.hstack([X_test_final, X_test_tfidf.toarray()])

print(f"Final feature count is now: {X_train_final_v2.shape[1]}")

In [ ]:
!pip install optuna

In [ ]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import numpy as np

# This assumes your final, most feature-rich dataset (X_train_final) is loaded.
# And your target variable y_train is also loaded.
# And the 'smape' function is defined.

def objective(trial):
    """
    This function defines one experiment for Optuna to run.
    """
    # 1. Suggest a set of hyperparameters for this trial
    params = {
        'objective': 'regression_l1',
        'metric': 'rmse',
        'n_estimators': 2000,
        'verbosity': -1,
        'n_jobs': -1,
        'seed': 42,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
    }

    # 2. Train a model with these params on a validation set
    # We use a simple split here to make each trial fast.
    X_train_sub, X_val, y_train_sub, y_val = train_test_split(
        X_train_final, y_train, test_size=0.2, random_state=42
    )

    model = lgb.LGBMRegressor(**params) # The ** unpacks the dictionary of params
    model.fit(X_train_sub, y_train_sub,
              eval_set=[(X_val, y_val)],
              callbacks=[lgb.early_stopping(100, verbose=False)])

    # 3. Make predictions and return the SMAPE score
    preds = model.predict(X_val)
    preds = np.expm1(preds) # Convert back from log scale
    y_val_prices = np.expm1(y_val) # Convert true values back too

    score = smape(y_val_prices, preds)
    return score

In [ ]:
# Create a study. We want to 'minimize' the SMAPE score.
study = optuna.create_study(direction='minimize')

# Start the optimization process. It will run the 'objective' function 50 times.
# This will take some time.
study.optimize(objective, n_trials=50)

# --- After it's done, get the results ---
best_params_lgbm = study.best_params
best_score = study.best_value

print("\n" + "="*50)
print(f"Best SMAPE score found: {best_score:.4f}%")
print("Best parameters found:")
print(best_params_lgbm)
print("="*50)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# This assumes your train_df and test_df are loaded

print("Creating powerful TF-IDF features...")

# We will look for the 2000 most important single words, word pairs, and triplets
tfidf = TfidfVectorizer(
    ngram_range=(1, 3), # Use single words, pairs, and triplets
    max_features=2000,  # Limit to the top 2000 most important phrases
    stop_words='english'
)

# Fit on the training data and transform both train and test data
X_train_tfidf = tfidf.fit_transform(train_df['catalog_content'].fillna(''))
X_test_tfidf = tfidf.transform(test_df['catalog_content'].fillna(''))

print("✅ TF-IDF features created successfully.")

In [ ]:
# Convert the sparse TF-IDF matrix to a dense array so we can combine it
X_train_tfidf_dense = X_train_tfidf.toarray()
X_test_tfidf_dense = X_test_tfidf.toarray()

# --- Rebuild your FINAL dataset, now with TF-IDF features ---
X_train_ultimate = np.hstack([X_train_final, X_train_tfidf_dense])
X_test_ultimate = np.hstack([X_test_final, X_test_tfidf_dense])

print(f"Your ultimate feature count is now: {X_train_ultimate.shape[1]}")

In [ ]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

# This assumes your ultimate datasets are in memory:
# X_train_ultimate, X_test_ultimate, and y_train

# ===================================================================
# Part 1: Re-Train LightGBM on the Ultimate Dataset
# ===================================================================
print("--- Training LightGBM on Ultimate Features ---")
NFOLDS = 5
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=42)

oof_predictions_lgbm_ult = np.zeros(X_train_ultimate.shape[0])
test_predictions_lgbm_ult = np.zeros(X_test_ultimate.shape[0])

for fold, (train_index, val_index) in enumerate(kf.split(X_train_ultimate, y_train)):
    print(f"LGBM Fold {fold+1}/{NFOLDS}")
    x_train_fold, x_val_fold = X_train_ultimate[train_index], X_train_ultimate[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    model_lgbm = lgb.LGBMRegressor(
        objective='regression_l1', n_estimators=2000, learning_rate=0.05,
        num_leaves=32, n_jobs=-1, seed=42
    )
    model_lgbm.fit(x_train_fold, y_train_fold,
                   eval_set=[(x_val_fold, y_val_fold)],
                   eval_metric='rmse',
                   callbacks=[lgb.early_stopping(100, verbose=False)])

    oof_predictions_lgbm_ult[val_index] = model_lgbm.predict(x_val_fold)
    test_predictions_lgbm_ult += model_lgbm.predict(X_test_ultimate) / NFOLDS

# ===================================================================
# Part 2: Re-Train XGBoost on the Ultimate Dataset
# ===================================================================
print("\n--- Training XGBoost on Ultimate Features ---")
oof_predictions_xgb_ult = np.zeros(X_train_ultimate.shape[0])
test_predictions_xgb_ult = np.zeros(X_test_ultimate.shape[0])

for fold, (train_index, val_index) in enumerate(kf.split(X_train_ultimate, y_train)):
    print(f"XGBoost Fold {fold+1}/{NFOLDS}")
    x_train_fold, x_val_fold = X_train_ultimate[train_index], X_train_ultimate[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    model_xgb = xgb.XGBRegressor(
        objective='reg:squarederror', n_estimators=2000, learning_rate=0.05,
        tree_method='hist', device='cuda', early_stopping_rounds=100,
        n_jobs=-1, seed=42
    )
    model_xgb.fit(x_train_fold, y_train_fold,
                  eval_set=[(x_val_fold, y_val_fold)], verbose=False)

    oof_predictions_xgb_ult[val_index] = model_xgb.predict(x_val_fold)
    test_predictions_xgb_ult += model_xgb.predict(X_test_ultimate) / NFOLDS

# ===================================================================
# Part 3: Calculate Scores and Create the Final Ensemble
# ===================================================================

# --- Define the SMAPE function ---
def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

# --- Get true prices ---
actual_prices = np.expm1(y_train)

# --- Calculate individual model scores ---
score_lgbm_ult = smape(actual_prices, np.expm1(oof_predictions_lgbm_ult))
score_xgb_ult = smape(actual_prices, np.expm1(oof_predictions_xgb_ult))

# --- Create the ensemble prediction by averaging ---
oof_predictions_ensemble_ult = (oof_predictions_lgbm_ult * 0.5) + (oof_predictions_xgb_ult * 0.5)
score_ensemble_ult = smape(actual_prices, np.expm1(oof_predictions_ensemble_ult))

print("\n" + "="*50)
print(f"New LightGBM Score:   {score_lgbm_ult:.4f}%")
print(f"New XGBoost Score:    {score_xgb_ult:.4f}%")
print(f"ULTIMATE ENSEMBLE SCORE: {score_ensemble_ult:.4f}%")
print("="*50)

# ===================================================================
# Part 4: Create the Final Submission File
# ===================================================================

# --- Blend the test predictions ---
test_predictions_ensemble_ult = (test_predictions_lgbm_ult * 0.5) + (test_predictions_xgb_ult * 0.5)
final_predictions = np.expm1(test_predictions_ensemble_ult)

# --- Create the DataFrame ---
submission_df_ultimate = pd.DataFrame({
    'sample_id': test_df['sample_id'],
    'price': final_predictions
})
submission_df_ultimate['price'] = submission_df_ultimate['price'].clip(lower=0.01)

# --- Save the file ---
submission_df_ultimate.to_csv('test_out_ultimate.csv', index=False)

print("\n🎉 Final submission file 'test_out_ultimate.csv' created successfully!")
print("Here's a preview:")
print(submission_df_ultimate.head())

In [ ]:
# ===================================================================
# Step 1: Setup - All Necessary Imports
# ===================================================================
import pandas as pd
import numpy as np
import re
import os
import lightgbm as lgb
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import uniform, randint
from tqdm import tqdm

print("✅ All libraries imported successfully.")

# ===================================================================
# Step 2: Configuration & Data Loading
# ===================================================================

# --- ⚠️ IMPORTANT: EDIT THIS PATH ---
# Set the path to the folder containing all your data files (.csv and .npy).
# For Kaggle: '/kaggle/input/your-dataset-name/'
data_path = '/kaggle/input/challenge'

try:
    print("Loading core CSV and embedding files...")
    train_df = pd.read_csv(os.path.join(data_path, '/kaggle/input/challenge/train.csv'))
    test_df = pd.read_csv(os.path.join(data_path, '/kaggle/input/challenge/test.csv'))
    y_train = np.log1p(train_df['price'])
    
    train_text_embeddings = np.load(os.path.join(data_path, 'train_text_embeddings.npy'))
    test_text_embeddings = np.load(os.path.join(data_path, 'test_text_embeddings.npy'))
    train_image_embeddings = np.load(os.path.join(data_path, 'train_image_embeddings.npy'))
    test_image_embeddings = np.load(os.path.join(data_path, 'test_image_embeddings.npy'))
    print("✅ All data loaded successfully.")
except FileNotFoundError as e:
    print(f"--- ERROR: A file was not found in '{data_path}'. Please check the path. ---")
    print(f"Missing file: {e.filename}")
    assert False, "Execution stopped."

# ===================================================================
# Step 3: Create All Features (Parsed + TF-IDF)
# ===================================================================
print("\n--- Creating all features ---")
# (Functions for feature engineering)
def find_brand(text):
    text = str(text).lower(); brands = ['apple', 'samsung', 'sony', 'nike', 'dell', 'hp'];
    for brand in brands:
        if brand in text: return brand
    return 'unknown'
def check_for_bulk(text):
    text = str(text).lower(); bulk_keywords = ['kit', 'pallet', 'case', 'bucket', 'pack'];
    for keyword in bulk_keywords:
        if keyword in text: return 1
    return 0
def extract_ipq(text):
    match = re.search(r'(?:IPQ|Pack of|Count)[\s:]*(\d+)', str(text), re.IGNORECASE)
    return int(match.group(1)) if match else 1

for df in [train_df, test_df]:
    df['brand'] = df['catalog_content'].apply(find_brand)
    df['is_bulk'] = df['catalog_content'].apply(check_for_bulk)
    df['item_quantity'] = df['catalog_content'].apply(extract_ipq)

train_brands_encoded = pd.get_dummies(train_df['brand'], prefix='brand'); test_brands_encoded = pd.get_dummies(test_df['brand'], prefix='brand')
train_brands_encoded, test_brands_encoded = train_brands_encoded.align(test_brands_encoded, join='left', axis=1, fill_value=0)
parsed_features_train = train_df[['item_quantity', 'is_bulk']].values
parsed_features_test = test_df[['item_quantity', 'is_bulk']].values

tfidf = TfidfVectorizer(ngram_range=(1, 3), max_features=2000, stop_words='english')
X_train_tfidf = tfidf.fit_transform(train_df['catalog_content'].fillna('')).toarray()
X_test_tfidf = tfidf.transform(test_df['catalog_content'].fillna('')).toarray()

# --- Combine all features into one final matrix ---
X_train_final = np.hstack([parsed_features_train, train_brands_encoded.values, train_text_embeddings, train_image_embeddings, X_train_tfidf])
X_test_final = np.hstack([parsed_features_test, test_brands_encoded.values, test_text_embeddings, test_image_embeddings, X_test_tfidf])
print(f"✅ Final dataset created. Total features: {X_train_final.shape[1]}")

# ===================================================================
# Step 4: Hyperparameter Tuning with Random Search ⚙️
# ===================================================================
print("\n--- Starting Hyperparameter Tuning with Random Search ---")
print("This will test 50 random combinations to find the best settings. This will take time.")

# 1. Define the parameter "distributions" to sample from
param_dist = {
    'learning_rate': uniform(0.01, 0.09), # Randomly pick a float between 0.01 and 0.1
    'num_leaves': randint(30, 100),       # Randomly pick an integer between 30 and 100
    'max_depth': randint(8, 25),
    'subsample': uniform(0.6, 0.4),       # Range is start, (end-start)
    'colsample_bytree': uniform(0.6, 0.4)
}

# 2. Set up and run the search
random_search = RandomizedSearchCV(
    estimator=lgb.LGBMRegressor(objective='regression_l1', n_estimators=2000, n_jobs=-1, seed=42),
    param_distributions=param_dist,
    n_iter=50, # Number of random combinations to try
    scoring='neg_root_mean_squared_error',
    cv=3, # Use 3-fold CV for speed during tuning
    verbose=1,
    random_state=42
)
random_search.fit(X_train_final, y_train)

# 3. Get the best parameters found
best_params = random_search.best_params_
print("\n✅ Random Search complete!")
print("Best parameters found:")
print(best_params)

# ===================================================================
# Step 5: Final Training with Best Parameters 🚀
# ===================================================================
print("\n--- Training final model using the best parameters found ---")

# Add the fixed parameters to the dictionary found by the search
best_params['objective'] = 'regression_l1'
best_params['n_estimators'] = 2000
best_params['n_jobs'] = -1
best_params['seed'] = 42

NFOLDS = 5
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=42)
oof_predictions = np.zeros(len(train_df))
test_predictions = np.zeros(len(test_df))

for fold, (train_idx, val_idx) in enumerate(tqdm(kf.split(X_train_final, y_train), total=NFOLDS, desc="Final Training Folds")):
    X_train, X_val = X_train_final[train_idx], X_train_final[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    # Initialize the model WITH THE BEST PARAMETERS
    model = lgb.LGBMRegressor(**best_params)
    model.fit(X_train, y_train_fold,
              eval_set=[(X_val, y_val_fold)],
              eval_metric='rmse',
              callbacks=[lgb.early_stopping(100, verbose=False)])
    
    oof_predictions[val_idx] = model.predict(X_val)
    test_predictions += model.predict(X_test_final) / NFOLDS

# ===================================================================
# Step 6: Evaluate and Create Submission 📄
# ===================================================================
def smape(y_true, y_pred):
    num = np.abs(y_pred - y_true); den = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = np.where(den == 0, 0, num / den)
    return np.mean(ratio) * 100

final_score = smape(train_df['price'].values, np.expm1(oof_predictions))
print("\n" + "="*50)
print(f"Final Estimated OOF SMAPE Score: {final_score:.4f}%")
print("="*50)

final_predictions = np.expm1(test_predictions)
submission_df = pd.DataFrame({'sample_id': test_df['sample_id'], 'price': final_predictions})
submission_df['price'] = submission_df['price'].clip(lower=0.01)
submission_df.to_csv('submission_tuned.csv', index=False)

print("\n🎉 Final submission file 'submission_tuned.csv' created successfully!")
print(submission_df.head())